In [18]:
import pandas as pd
import numpy as np
import random as rnd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bays
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier #stochastic gradient descent
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

In [19]:
data = pd.read_csv('/Users/aakash_sivasankar/Main/College/Sr_2/ECE_40875/OSS_Senior_Design/founder_V3_withsurvey.csv')

# Replace detailed role variable with binary variable which represents if the person is founder or not 
data['Primary Job Title'] = data['Founder']

# Drop variables not to be used in our models
data = data.drop(['Full Name','Bio','Primary Job Title'], axis=1)

data.head()

KeyError: 'Founder'

In [9]:
# Convert gender variable (0 or 1 or 2) to integers
data['Gender'] = data['Gender'].astype(int)

data.shape

(18361, 29)

In [10]:
# Drop those rows that have 2 as Gender variable (these are the ones who do not wish to identify their genders)
data=data[(data.Gender==2) == False]

data.shape

(18230, 29)

In [11]:
# Changing the strings in the Headquarters Location column with their frequencies over the all dataset.
df1 = data['Headquarters Location '].value_counts().reset_index(name='count').rename(columns={'index': 'Headquarters Location '})

data = pd.merge(data,df1,on=['Headquarters Location '], how='left' )

data.head()


,Gender,Number of News Articles,Number of Founded Organizations,Number of Portfolio Companies,Number of Investments_x,Number of Partner Investments,Number of Lead Investments_x,Number of Exits_x,Number of Events_x,Categories,...,Persistence,ConvinceOthers,RelyOnOthers,Competition,AmSuccessfulEntrepreneur,StrongProfNetwork,WorkLifeBalance,NineFiveJob,Success,count
0,1,14445,7,11,21,0,9,6,2,"Innovation Management, Automotive, Electronics",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6535
1,1,100,2,1,1,0,0,0,1,"Innovation Management, Automotive, Electronics",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6535
2,1,19678,10,6,7,0,0,0,5,"Social Media, Social, Social Network",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6535
3,1,128,1,6,7,0,0,1,5,"Apps, Enterprise Software, Developer APIs, Fin...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6535
4,1,861,1,0,0,0,0,0,0,"Internet, Self-Storage, E-Commerce",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1935


In [12]:
data['Headquarters Location '] = data['count']

# Drop the frequencies column and two other columns that will not be used in the model.

data = data.drop(['count'], axis=1)

data = data.drop(['Categories','Closed Date'], axis=1)

data.head()


,Gender,Number of News Articles,Number of Founded Organizations,Number of Portfolio Companies,Number of Investments_x,Number of Partner Investments,Number of Lead Investments_x,Number of Exits_x,Number of Events_x,Headquarters Location,...,FearOfFailure,Persistence,ConvinceOthers,RelyOnOthers,Competition,AmSuccessfulEntrepreneur,StrongProfNetwork,WorkLifeBalance,NineFiveJob,Success
0,1,14445,7,11,21,0,9,6,2,6535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,100,2,1,1,0,0,0,1,6535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1,19678,10,6,7,0,0,0,5,6535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1,128,1,6,7,0,0,1,5,6535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1,861,1,0,0,0,0,0,0,1935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [13]:
data['Number of News Articles'] = data['Number of News Articles']/data['Number of News Articles'].max()

data['Number of Founded Organizations'] = data['Number of Founded Organizations']/data['Number of Founded Organizations'].max()

data['Number of Portfolio Companies'] = data['Number of Portfolio Companies']/data['Number of Portfolio Companies'].max()

data['Number of Investments_x'] = data['Number of Investments_x']/data['Number of Investments_x'].max()

data['Number of Partner Investments'] = data['Number of Partner Investments']/data['Number of Partner Investments'].max()

data['Number of Lead Investments_x'] = data['Number of Lead Investments_x']/data['Number of Lead Investments_x'].max()

data['Number of Exits_x'] = data['Number of Exits_x']/data['Number of Exits_x'].max()

data['Number of Events_x'] = data['Number of Events_x']/data['Number of Events_x'].max()

data['Number of Founders'] = data['Number of Founders']/data['Number of Founders'].max()

data['Headquarters Location '] = data['Headquarters Location ']/data['Headquarters Location '].max()

#Split the data to training and test datasets (0.8 and 0.2)

from sklearn.model_selection import train_test_split

X_data2 = data.iloc[:,:-1] 
Y_data2 = data['Success'] 
    
print("Feature vector shape=", X_data2.shape)
print("Class shape=", Y_data2.shape)

Feature vector shape= (18230, 26)
Class shape= (18230,)


In [14]:
x_train, x_test, y_train, y_test = train_test_split(X_data2, Y_data2, test_size = 0.2 , random_state=100)

In [17]:
# XGBoost

gradboost = xgb.XGBClassifier(n_estimators = 10000)
gradboost.fit(x_train, y_train)
y_pred_xgb = gradboost.predict(x_test)

acc_gradboost_train = round(gradboost.score(x_train, y_train) * 100, 2)
print("Train accuracy: ", acc_gradboost_train)

acc_gradboost_test = round(gradboost.score(x_test,y_test) * 100, 2)
print("Test accuracy: ", acc_gradboost_test)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Email: object, CompanyName: object

In [ ]:
# Calculate and plot average precision recall score

from sklearn.metrics import average_precision_score
y_pred = y_pred_xgb


average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_test, y_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.show()

In [ ]:
# Look at importnace of features for random forest

def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

plot_model_var_imp(gradboost, x_train, y_train)

In [ ]:
plt.show()

In [ ]:
# Define Roc curve

from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier


y_test = label_binarize(y_test, classes=[1,0])
y_train = label_binarize(y_train, classes=[1,0])

n_classes = y_train.shape[1]

ovr = OneVsRestClassifier(gradboost)

y_score = ovr.fit(x_train, y_train).predict_proba(x_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(tpr[i], fpr[i])

In [ ]:
# Plot  ROC 

for i in range(n_classes):
    plt.figure()
    plt.plot(tpr[i], fpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(x_train, y_train)
y_pred_rnf = random_forest.predict(x_test)
random_forest.score(x_train, y_train)

acc_random_forest_train = round(random_forest.score(x_train, y_train) * 100, 2)
print("Train accuracy: ",acc_random_forest_train)

acc_random_forest_test = round(random_forest.score(x_test,y_test) * 100, 2)
print("Test accuracy: ",acc_random_forest_test)

In [ ]:
# Calculate and plot average precision recall score


y_pred = y_pred_rnf
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.form# Plot  ROC 

for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()at(
      average_precision))

precision, recall, _ = precision_recall_curve(y_test, y_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.show()

In [ ]:
# Look at importnace of features for random forest

def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

plot_model_var_imp(random_forest, x_train, y_train)

plt.show()

In [ ]:
# Define Roc curve

y_test = label_binarize(y_test, classes=[1,0])
y_train = label_binarize(y_train, classes=[1,0])

n_classes = y_train.shape[1]

ovr = OneVsRestClassifier(random_forest)

y_score = ovr.fit(x_train, y_train).predict_proba(x_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(tpr[i], fpr[i])

# Plot  ROC curve

for i in range(n_classes):
    plt.figure()
    plt.plot(tpr[i], fpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()



In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred_log = logreg.predict(x_test)

acc_log_train = round(logreg.score(x_train, y_train) * 100, 2)
print("Train accuracy: ",acc_log_train)

acc_log_test = round(logreg.score(x_test,y_test) * 100, 2)
print("Test accuracy: ",acc_log_test)


In [ ]:
# Calculate and plot average precision recall score

y_pred = y_pred_log
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

precision, recall, _ = precision_recall_curve(y_test, y_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.show()

In [ ]:
def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

plot_model_var_imp(logreg, x_train, y_train)

plt.show()

